In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as pofs
import xlsxwriter
import warnings

warnings.filterwarnings('ignore')

In [2]:
stocks = pd.read_csv('./sp_500_stocks.csv')
token = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={token}'
data = requests.get(api_url).json()

In [6]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'Price-to-Earning Ratio', 'Number of Shares to Buy']

In [9]:
final_df = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings: 
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={token}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            final_df = final_df.append(pd.Series([symbol, 
                                       data[symbol]['quote']['latestPrice'], 
                                       data[symbol]['quote']['peRatio'], 
                                       'N/A'], index=my_columns), ignore_index=True)
        except:
            pass

In [11]:
final_df.sort_values(by='Price-to-Earning Ratio', ascending=True, inplace=True)
final_df = final_df[final_df['Price-to-Earning Ratio'] > 0]
final_df = final_df[:50]
final_df.reset_index(drop=True, inplace=True)

In [20]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('Not a Number')
        portfolio_size = input('Enter the size of your portfolio: ')

In [21]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [22]:
position_size = float(portfolio_size) / len(final_df.index)
for i in range(len(final_df.index)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df.loc[i, 'Price'])

In [23]:
final_df

,Ticker,Price,Price-to-Earning Ratio,Number of Shares to Buy
0,VTR,38.81,934.21,515
1,GPN,114.64,433.3,174
2,NOW,372.23,399.4,53
3,STZ,231.65,383.09,86
4,CRM,159.3,298.31,125
5,KSU,307.23,285.84,65
6,DAL,32.71,257.75,611
7,APTV,85.02,211.91,235
8,DXCM,98.18,204.46,203
9,ALB,259.91,119.13,76


In [67]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earning Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score',
]

rv_df = pd.DataFrame(columns=rv_columns)

In [68]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={token}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            gross_profit = data[symbol]['advanced-stats']['grossProfit']
            
            try:
                ev_to_ebitda = enterprise_value/ebitda
            except TypeError:
                ev_to_ebitda = np.NaN
                
            try:
                ev_to_gp = enterprise_value/gross_profit
            except TypeError:
                ev_to_gp = np.NaN 
            
            rv_df = rv_df.append(pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gp,
                'N/A',
                'N/A',
            ], index=rv_columns), ignore_index=True)
        except:
            pass

In [69]:
len(rv_df[rv_df.isnull().any(axis=1)].index)

21

In [70]:
rv_df.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earning Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

In [71]:
for col in ['Price', 'Price-to-Earning Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_df[col].fillna(rv_df[col].mean(), inplace=True)

In [72]:
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.06,N/A,29.50,N/A,7.80,N/A,5.8300,N/A,20.578313,N/A,11.026167,N/A,N/A
1,AAL,13.72,N/A,-4.82,N/A,-1.09,N/A,0.2284,N/A,106.578085,N/A,0.824370,N/A,N/A
2,AAP,173.18,N/A,20.30,N/A,3.69,N/A,0.9630,N/A,11.433411,N/A,2.352374,N/A,N/A
3,AAPL,148.96,N/A,23.90,N/A,41.55,N/A,6.2400,N/A,18.361671,N/A,14.207764,N/A,N/A
4,ABBV,147.28,N/A,20.89,N/A,17.51,N/A,4.4200,N/A,11.732776,N/A,7.964192,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,113.44,N/A,22.02,N/A,-3.75,N/A,4.8700,N/A,19.158053,N/A,8.693135,N/A,N/A
497,ZBH,109.57,N/A,99.82,N/A,1.96,N/A,3.2000,N/A,17.460372,N/A,5.450450,N/A,N/A
498,ZBRA,261.87,N/A,28.14,N/A,5.40,N/A,2.3600,N/A,14.758825,N/A,5.857592,N/A,N/A
499,ZION,48.55,N/A,5.40,N/A,1.50,N/A,2.1900,N/A,4.544861,N/A,2.074147,N/A,N/A


In [73]:
metrics = {
    'Price-to-Earning Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile',
}

In [74]:
for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[metric]] = pofs(rv_df[metric], rv_df.loc[row, metric]) / 100

In [75]:
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.06,N/A,29.50,0.750499,7.80,0.796407,5.8300,0.842315,20.578313,0.838323,11.026167,0.826347,N/A
1,AAL,13.72,N/A,-4.82,0.033932,-1.09,0.05988,0.2284,0.013972,106.578085,0.996008,0.824370,0.02994,N/A
2,AAP,173.18,N/A,20.30,0.516966,3.69,0.555888,0.9630,0.159681,11.433411,0.411178,2.352374,0.149701,N/A
3,AAPL,148.96,N/A,23.90,0.618762,41.55,0.978044,6.2400,0.858283,18.361671,0.766467,14.207764,0.918164,N/A
4,ABBV,147.28,N/A,20.89,0.53493,17.51,0.938124,4.4200,0.758483,11.732776,0.427146,7.964192,0.666667,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,113.44,N/A,22.02,0.56487,-3.75,0.055888,4.8700,0.804391,19.158053,0.790419,8.693135,0.722555,N/A
497,ZBH,109.57,N/A,99.82,0.976048,1.96,0.338323,3.2000,0.60479,17.460372,0.732535,5.450450,0.431138,N/A
498,ZBRA,261.87,N/A,28.14,0.726547,5.40,0.698603,2.3600,0.447106,14.758825,0.62475,5.857592,0.469062,N/A
499,ZION,48.55,N/A,5.40,0.087824,1.50,0.221557,2.1900,0.427146,4.544861,0.087824,2.074147,0.131737,N/A


In [76]:
from statistics import mean

for row in rv_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row, 'RV Score'] = mean(value_percentiles)

In [77]:
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.06,N/A,29.50,0.750499,7.80,0.796407,5.8300,0.842315,20.578313,0.838323,11.026167,0.826347,0.810778
1,AAL,13.72,N/A,-4.82,0.033932,-1.09,0.05988,0.2284,0.013972,106.578085,0.996008,0.824370,0.02994,0.226747
2,AAP,173.18,N/A,20.30,0.516966,3.69,0.555888,0.9630,0.159681,11.433411,0.411178,2.352374,0.149701,0.358683
3,AAPL,148.96,N/A,23.90,0.618762,41.55,0.978044,6.2400,0.858283,18.361671,0.766467,14.207764,0.918164,0.827944
4,ABBV,147.28,N/A,20.89,0.53493,17.51,0.938124,4.4200,0.758483,11.732776,0.427146,7.964192,0.666667,0.66507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,113.44,N/A,22.02,0.56487,-3.75,0.055888,4.8700,0.804391,19.158053,0.790419,8.693135,0.722555,0.587625
497,ZBH,109.57,N/A,99.82,0.976048,1.96,0.338323,3.2000,0.60479,17.460372,0.732535,5.450450,0.431138,0.616567
498,ZBRA,261.87,N/A,28.14,0.726547,5.40,0.698603,2.3600,0.447106,14.758825,0.62475,5.857592,0.469062,0.593214
499,ZION,48.55,N/A,5.40,0.087824,1.50,0.221557,2.1900,0.427146,4.544861,0.087824,2.074147,0.131737,0.191218


In [78]:
rv_df.sort_values(by='RV Score', ascending=True, inplace=True)
rv_df = rv_df[:50]
rv_df.reset_index(drop=True, inplace=True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,PVH,49.14,N/A,3.58,0.051896,0.6153,0.06986,0.3495,0.033932,4.019488,0.061876,0.857656,0.03992,0.051497
1,AIG,52.35,N/A,3.52,0.0499,0.9165,0.0998,0.7268,0.10978,2.068959,0.017964,0.720618,0.015968,0.058683
2,KSS,28.80,N/A,5.67,0.08982,0.7569,0.081836,0.1843,0.005988,4.143051,0.065868,1.070913,0.055888,0.05988
3,F,12.20,N/A,4.14,0.055888,1.1200,0.132735,0.3275,0.02994,3.234898,0.031936,1.669750,0.097804,0.069661
4,GM,34.66,N/A,6.68,0.113772,0.7936,0.083832,0.3866,0.045908,2.015744,0.015968,1.545502,0.091816,0.070259
5,C,44.28,N/A,5.70,0.092814,0.4733,0.065868,1.0000,0.171657,3.367341,0.035928,1.003118,0.051896,0.083633
6,SYF,31.73,N/A,4.76,0.070858,1.2000,0.145709,0.9035,0.143713,2.889411,0.023952,0.955220,0.045908,0.086028
7,COF,98.82,N/A,4.34,0.05988,0.7178,0.073852,1.1100,0.203593,3.008446,0.027944,1.149487,0.065868,0.086228
8,UAA,6.95,N/A,8.81,0.157685,1.3400,0.177645,0.4928,0.065868,2.690887,0.01996,0.626047,0.011976,0.086627
9,HPQ,25.70,N/A,4.45,0.063872,-11.3300,0.043912,0.4045,0.053892,5.692490,0.125749,2.499641,0.163673,0.09022


In [79]:
portfolio_input()

Enter the size of your portfolio: 2500000


In [80]:
position_size = float(portfolio_size) / len(rv_df.index)
for i in range(len(rv_df.index)):
    rv_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_df.loc[i, 'Price'])

In [81]:
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,PVH,49.14,1017,3.58,0.051896,0.6153,0.06986,0.3495,0.033932,4.019488,0.061876,0.857656,0.03992,0.051497
1,AIG,52.35,955,3.52,0.0499,0.9165,0.0998,0.7268,0.10978,2.068959,0.017964,0.720618,0.015968,0.058683
2,KSS,28.80,1736,5.67,0.08982,0.7569,0.081836,0.1843,0.005988,4.143051,0.065868,1.070913,0.055888,0.05988
3,F,12.20,4098,4.14,0.055888,1.1200,0.132735,0.3275,0.02994,3.234898,0.031936,1.669750,0.097804,0.069661
4,GM,34.66,1442,6.68,0.113772,0.7936,0.083832,0.3866,0.045908,2.015744,0.015968,1.545502,0.091816,0.070259
5,C,44.28,1129,5.70,0.092814,0.4733,0.065868,1.0000,0.171657,3.367341,0.035928,1.003118,0.051896,0.083633
6,SYF,31.73,1575,4.76,0.070858,1.2000,0.145709,0.9035,0.143713,2.889411,0.023952,0.955220,0.045908,0.086028
7,COF,98.82,505,4.34,0.05988,0.7178,0.073852,1.1100,0.203593,3.008446,0.027944,1.149487,0.065868,0.086228
8,UAA,6.95,7194,8.81,0.157685,1.3400,0.177645,0.4928,0.065868,2.690887,0.01996,0.626047,0.011976,0.086627
9,HPQ,25.70,1945,4.45,0.063872,-11.3300,0.043912,0.4045,0.053892,5.692490,0.125749,2.499641,0.163673,0.09022


In [82]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_df.to_excel(writer, 'Value Strategy', index = False)

In [83]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
float_format = writer.book.add_format(
    {
        'num_format': '0.0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
percentile_format = writer.book.add_format(
    {
        'num_format': '0.00%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [84]:
column_formats = {
   'A': ['Ticker', string_format],
   'B': ['Price', dollar_format],
   'C': ['Number of Shares to Buy', integer_format],
   'D': ['Price-to-Earning Ratio', float_format],
   'E': ['PE Percentile', percentile_format],
   'F': ['Price-to-Book Ratio', float_format],
   'G': ['PB Percentile', percentile_format],
   'H': ['Price-to-Sales Ratio', float_format],
   'I': ['PS Percentile', percentile_format],
   'J': ['EV/EBITDA', float_format],
   'K': ['EV/EBITDA Percentile', percentile_format],
   'L': ['EV/GP', float_format],
   'M': ['EV/GP Percentile', percentile_format],
   'N': ['RV Score', percentile_format],
}

for col in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{col}:{col}', 25, column_formats[col][1])
    writer.sheets['Value Strategy'].write(f'{col}1', column_formats[col][0], column_formats[col][1])
writer.save()

In [85]:
writer.close()